# M3 Reconstruction
### Training a Neural Network to identify the best M3 Combo
By Zach Shelton
5/13/21


In [1]:
#NanoAOD HackSchema
#Solution from Danny Noonan

In [365]:
from __future__ import print_function, division
import uproot
import numpy as np
#Make sure to install both old awkward0 and new awkward1(referred to now as awkward)
import awkward1 as ak
from coffea.nanoevents import NanoAODSchema,NanoEventsFactory
import coffea
from uproot3_methods import TLorentzVectorArray
import uproot3_methods
import numpy as np
import coffea.hist as hist
import matplotlib.pyplot as plt
import awkward
class HackSchema(NanoAODSchema):
    def __init__(self, base_form):
        base_form["contents"].pop("Muon_fsrPhotonIdx", None)
        base_form["contents"].pop("Electron_photonIdx", None)
        super().__init__(base_form)
files ="TTbarPowheg_Semilept_Skim_NanoAOD_1of21.root"
import coffea.processor as processor
from pprint import pprint
file=uproot.open(files)
from pprint import pprint

## Note: It seems the Jet columns are sorted from greatest p_t to smallest p_t
Feel free to test, but it seems to be my observation, choosing the 1st, 2nd or 3rd jet via index should remove the issue of it being a coffea sorting artifact or procedure

In [2]:
#Now lets redo with the cuts detailed by CMS Draft Analysis
#https://drive.google.com/file/d/1XEOLyZ-Q1HdEQY379RpyyQkOF1Q8KlsL/view
events =NanoEventsFactory.from_root(files,schemaclass=HackSchema).events()

In [3]:
#Condensing_all Lepton_cuts
tight_jets=events.Jet
jetSel = ak.num(tight_jets[((tight_jets.pt>30)&(tight_jets.eta<2.4)&(tight_jets.eta>-2.4))],axis=1)>=3
jetSelection=(jetSel&(ak.num(tight_jets.btagCSVV2>.4184)>=1))
tight_muons = events.Muon
muonsel=ak.num(tight_muons[((tight_muons.pt>30)&(abs(tight_muons.eta)<2.4))],axis=1)==1
tight_electrons= events.Electron
electronsel=ak.num(tight_electrons[((tight_electrons.pt>35)&(abs(tight_electrons.eta)<2.4))],axis=1)==1
leptonsel=(muonsel|electronsel)
jetlepselmask = (jetSelection&leptonsel)
final=events[jetlepselmask]

In [ ]:
jets = ak.zip({"pt":final.Jet.pt[:,0:8],"eta":final.Jet.eta[:,0:8],"phi":final.Jet.phi[:,0:8],"mass":final.Jet.mass[:,0:8],"btag":final.Jet.btagCSVV2[:,0:8]})
jet_kinematics=ak.zip({"pt":final.Jet.pt[:,0:8],"eta":final.Jet.eta[:,0:8],"phi":final.Jet.phi[:,0:8],"mass":final.Jet.mass[:,0:8]},with_name="PtEtaPhiMLorentzVector")

jets.fields

In [ ]:
#First cut Combos without b-tagged
#This will become my data tensor I pass to a Neural Net, pending additions of more(the combinations):
comb= ak.combinations(jets,n=3,axis=1,highlevel=1)
truthcomb=ak.combinations(final.GenJet.partonFlavour[:,0:8],n=3,axis=1)

In [103]:
#G = Global index of particles - Artifact from Coffea/NanoAOD Tools
final.GenPart.mass

<Array [[0, 0, 172, 170, 0, ... 0, 0, 0, 0, 0]] type='1381189 * var * float32[pa...'>

In [27]:
genPart = events.GenPart
tops = genPart[abs(genPart.pdgId)==6]
tops = tops[tops.hasFlags('isLastCopy')]
tDecay = tops.distinctChildren
tDecay = tDecay[tDecay.hasFlags('isLastCopy')]
t_Events=tDecay[abs(tDecay.pdgId)==5]
W = tDecay[abs(tDecay.pdgId)==24]
W = W[W.hasFlags('isLastCopy')]
WDecay = W.distinctChildren
WDecay = WDecay[WDecay.hasFlags('isLastCopy')]
#t_events is the lone bottom, W_events is the -> two jets

In [28]:
isHadronic = ak.all(abs(WDecay.pdgId)<10,axis=-1)
pprint(isHadronic[1])

<Array [[True], [False]] type='2 * var * ?bool'>


In [376]:
final.Jet[0]

<JetArray [Jet, Jet, Jet, Jet, Jet] type='5 * jet'>

In [377]:
t_Events[0][0].nearest(final.Jet[0])

ValueError: cannot broadcast records because keys don't match:
    childrenIdxG, distinctChildrenIdxG, distinctParentIdxG, eta, genPartIdxMother, genPartIdxMotherG, mass, pdgId, phi, pt, status, statusFlags
    area, bRegCorr, bRegRes, btagCMVA, btagCSVV2, btagDeepB, btagDeepC, btagDeepFlavB, btagDeepFlavC, chEmEF, chHEF, cleanmask, electronIdx1, electronIdx1G, electronIdx2, electronIdx2G, electronIdxG, eta, genJetIdx, genJetIdxG, hadronFlavour, jercCHF, jercCHPUF, jetId, mass, muEF, muonIdx1, muonIdx1G, muonIdx2, muonIdx2G, muonIdxG, muonSubtrFactor, nConstituents, nElectrons, nMuons, neEmEF, neHEF, partonFlavour, phi, pt, puId, qgl, rawFactor

(https://github.com/scikit-hep/awkward-1.0/blob/0.4.5/src/awkward1/_util.py#L915)

In [288]:
w_mins=[]
for ws in W_Events[0]:
    #print(ws)
    if len(ws)>0:
        for w in ws:
            for q in w:
                q_mins=[]
                print(q.pdgId)
                delta_rs=[]
                for j in final.Jet[0]:
                    dr=q.delta_r(j)
                    delta_rs.append(dr)
            q_mins.append(np.argmin(delta_rs))
        w_mins.append(q_mins)
    else:
        w_mins.append(None)

2
-1


In [287]:
print(w_mins)

[[3], None]


In [42]:
#(-)6->5+W->
#(-)6->5+W->
#1-6 are quarks
#11-16 are leptons+neutrinons
#20's are bosons
#25 higgs
#genPart.hasFlags('isLastCopy') gets rid of copies

______________________________________________________________________________
# Running Notes and questions
- Losing Data because of arbitrary 8 jet cut?

## Notes for Zach to keep up
- Jet TightID
- More 
- Particle Values
- Delta R

Wrap into Coffea Executor
Seperate values, weights and triggers
read these into tensors for Keras
Best ML Algorithm?
- Deep Neural Net
- Iterative Boosted Tree
    - They are fast
    - External ML algorithm modifies BDT parameters
- Combine old processes together
Should I use Keras or PyTorch

## Sources/Further Reading:

CMS Draft Analysis Note(FIT, AAS, Wayne State, NISTR): https://drive.google.com/file/d/1XEOLyZ-Q1HdEQY379RpyyQkOF1Q8KlsL/view

Invariant Mass Wikipedia Article: https://en.wikipedia.org/wiki/Invariant_mass

Measurement of the semileptonic tt + γ production cross-section in pp collisions at √s = 8 Tev: https://arxiv.org/pdf/1706.08128.pdf

Coffea Documentation: https://coffeateam.github.io/coffea/

Awkward Array Documentation: https://awkward-array.readthedocs.io/en/latest/

AOD DATA GUIDE: https://twiki.cern.ch/twiki/bin/view/CMSPublic/SWGuideAodDataTable